# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### 先搜尋全部的電影代號(ID)資訊

In [2]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')

#<select id="sbox_mid" name="movie_id" class="movie_name">
html = soup.find("select", attrs={'name':'movie_id'})

#<option value="10420" data-name="鹿皮奇談">鹿皮奇談</option>
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')}) #compile正則表達式

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"])) # %s 使用str()將字串輸出,

Movie: 兔嘲男孩, ID: 10283
Movie: 海上鋼琴師4K數位修復, ID: 10425
Movie: 鹿皮奇談, ID: 10420
Movie: 大約在冬季, ID: 10408
Movie: 倒數計死, ID: 10390
Movie: 吹哨人, ID: 10381
Movie: 幸福試用期, ID: 10374
Movie: 竊竊屍語, ID: 10365
Movie: 舞力決戰, ID: 10344
Movie: 潛艦追緝, ID: 10333
Movie: 霹靂嬌娃, ID: 10124
Movie: 白頭山：半島浩劫, ID: 10354
Movie: CATS貓, ID: 10166
Movie: 金魚, ID: 10424
Movie: 海上花, ID: 10376
Movie: 奪魂露營車, ID: 10366
Movie: 怨靈古堡, ID: 10356
Movie: 添好孕, ID: 10332
Movie: 外出偷馬, ID: 10310
Movie: 茱蒂, ID: 10112
Movie: 燃燒女子的畫像, ID: 10069
Movie: 葉問4：完結篇, ID: 9890
Movie: STAR WARS : 天行者的崛起, ID: 10225
Movie: 復仇母親, ID: 10392
Movie: 風暴過後, ID: 10387
Movie: 南方車站的聚會, ID: 10380
Movie: 祭旗, ID: 10373
Movie: 黑熊來了, ID: 10357
Movie: 黑計畫, ID: 10351
Movie: 假面騎士劇場版ZI-O Over Quartzer, ID: 10331
Movie: 8級警戒, ID: 10323
Movie: 為你存在的每一天, ID: 10297
Movie: 黑色聖誕節, ID: 10291
Movie: 熱氣球飛行家, ID: 10257
Movie: 大說謊家, ID: 10122
Movie: 特約經紀公司, ID: 10364
Movie: 人間失格：太宰治與他的3個女人, ID: 10287
Movie: 呆萌特務, ID: 10264
Movie: 去年聖誕節, ID: 10256
Movie: 野蠻遊戲：全面晉級, ID: 10134
Movie:

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [14]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = 10425

In [15]:
## https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater?movie_id=10425
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header ,request header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
# print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 台中, 代號(area_id): 2
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [16]:
# 指定放映地區
area_id = 28

In [17]:
## https://movies.yahoo.com.tw/movietime_result.html?movie_id=10425&area_id=28
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')

'''
<input id="date_0" name="date" type="radio" value="2019-12-31" class="calendar_checked jq_calendar_checked gabtn" 
data-ga="['電影頁_時刻表', '電影頁_時刻表_日曆', '海上鋼琴師4K數位修復']" data-href="https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie" 
data-movie="10425" data-area="28" checked="">
<label for="date_0" class="picker_label">
  <p class="month">十二月</p>
  <h3 class="day">31</h3>
  <p class="week">今天</p>
</label>
</li>

#<input id="date_0" name="date"
#<input id="date_1" name="date"
#<input id="date_2" name="date"
'''
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

一月 1
一月 2
一月 3
一月 4
一月 5


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [18]:
# 選定要觀看的日期
date = "2020-01-01"

In [30]:
## https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie?movie_id=10425&date=2019-12-31&area_id=28&theater_id=&datetime=&movie_type_id=
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")}) ## 把 電影院 擷取出來
html
# <ul class="area_time _c jq_area_time" data-theater_name="國賓影城(台北長春廣場)"
#   data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=29" 
#   data-theater_url="http://www.ambassador.com.tw/" id="theater_id_29">

[<ul class="area_time _c jq_area_time" data-theater_name="國賓影城(台北長春廣場)" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=29" data-theater_url="http://www.ambassador.com.tw/" id="theater_id_29">
 <li class="adds">
 <a href="https://movies.yahoo.com.tw/theater_result.html/id=29">國賓影城(台北長春廣場)</a>
 <span>02-25155755</span>
 </li>
 <li class="taps">
 <span class="tapR">數位</span> </li>
 <li class="time _c">
 <div class="input_picker jq_input_picker">
 <input class="gabtn" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '海上鋼琴師4K數位修復']" data-movie_date="01.01" data-movie_time="11:00" data-movie_title="海上鋼琴師4K數位修復" data-movie_type="數位" data-news="news" id="72685403" name="schedule_list" type="radio" value="2020-01-01 11:00:00"/>
 <label class="" for="72685403">11:00</label>
 <input class="gabtn" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '海上鋼琴師4K數位修復']" data-movie_date="01.01" data-movie_time="13:15" data-movie_title="海上鋼琴師4K數位修復" data-movie_type="數位" data-news="news" id="72685404" name="s

In [33]:
'''
  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
    Your code here.
'''
#==2========================================================   
# 列印電影院名稱、影片放映類型以及時間表
for item in html:
#     print(item)
    theater_name = item.a.string #電影院
    #<a href="https://movies.yahoo.com.tw/theater_result.html/id=29">國賓影城(台北長春廣場)</a>
    print("電影院: ",theater_name)
    
    movie_type = item.find("span",class_="tapR").string  #放映類型
    # <span class="tapR">數位</span>
    print("放映類型: ",movie_type)
    
    movie_time_list = item.find_all("input",class_="gabtn") # 找 時間
    for i in range(len(movie_time_list)):
        movie_time = movie_time_list[i]['value'] #時間
        #print("%s %s %s" % (theater_name, movie_type,movie_time)) #一次顯示
        print(movie_time)
    print("-------------------------------------------")    
# #==1========================================================  
# for p in html:
#     print("----------------------------------------------------------------------")
#     print("電影院:  %s" % p['data-theater_name'])
#     #---找 放映類型 ---------------------
#     movie_info = p.find_all("li", attrs={"class":"taps"}) #數位 
# #     <li class="taps">
# #      <span class="tapR">數位</span>
# #     </li>
      
#     taps = []
#     for info in movie_info:
#         movie_type = info.find_all("span")
#         tap = []
#         for mtype in movie_type:
#             tap.append(mtype.text)
#         taps.append(' '.join(tap)) #join 將序列中的元素以指定的字符連接生成一個新的字符串。
#     print(taps)
#     #-----------------------
#     movie_info = p.find_all("div", attrs={"class":"input_picker jq_input_picker"}) #時間表
#     for info in range(len(movie_info)):
#         print("放映類型:  %s" % taps[info]) #數位
#         schedule_list = movie_info[info].find_all("input", attrs={"class":"gabtn"}) #時間
#         for schedule in schedule_list:
#             print(schedule['value'])

        
# ----------------------------------------------------------------------
# 電影院:  台北美麗華大直影城
# 放映類型:  數位
# 2019-08-21 09:00:00
# 2019-08-21 11:10:00
# 2019-08-21 13:15:00
# 2019-08-21 15:20:00
# 2019-08-21 19:30:00
# 2019-08-21 21:40:00
# 2019-08-21 22:30:00

電影院:  國賓影城(台北長春廣場)
放映類型:  數位
2020-01-01 11:00:00
2020-01-01 13:15:00
2020-01-01 15:30:00
2020-01-01 17:45:00
2020-01-01 20:00:00
2020-01-01 22:15:00
-------------------------------------------
電影院:  台北真善美劇院
放映類型:  數位
2020-01-01 20:50:00
-------------------------------------------
電影院:  台北信義威秀影城
放映類型:  數位
2020-01-01 20:05:00
-------------------------------------------
電影院:  光點華山電影館
放映類型:  數位
2020-01-01 10:30:00
2020-01-01 15:30:00
2020-01-01 19:30:00
-------------------------------------------
電影院:  誠品電影院
放映類型:  數位
2020-01-01 14:20:00
2020-01-01 21:30:00
-------------------------------------------
電影院:  喜樂時代影城南港店
放映類型:  數位
2020-01-01 19:30:00
-------------------------------------------
